# Project: Anomaly Detection in Banking Transactions Using K-means


## Objective

Use K-means to automatically identify unusual behaviors in a set of synthetic transactions. The clusters model different behavior patterns, and anomalies can be detected by observing points that are too far from the cluster centers.

In [186]:
# Importing necessary modules for data processing clustering and visualization

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [187]:
# This function is responsible for loading a dataset from a CSV file

def load_data(file_path: str) -> pd.DataFrame:
    """
    Load the transaction dataset from the given CSV file.

    :param file_path: Path to the CSV file
    :return: Pandas DataFrame with the transaction data
    """

    data_as_dataframe = pd.read_csv(file_path)
    return data_as_dataframe

In [188]:
# This function is used to select the useful variables for the program. 
# I chose to remove global_id, sender_id, and date because they are only useful for identifying transactions, 
# but this information will not help us. 
# Unless we consider specific days (e.g., Fridays) associated with fraud, which we won't analyze here 
# since we are not treating this as a time series problem.

def select_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Select relevant numerical features for clustering.

    :param df: Full transaction DataFrame
    :return: DataFrame with selected features only
    """

    select_features_df = df.drop(columns=['global_id', 'sender_id', 'receiver_id', 'date']).copy()
    return select_features_df

In [189]:
# This function is used during the preprocessing step to standardize datasets. 
# Its goal is to mitigate bias that could arise from large variations in feature magnitudes within the dataset.

def normalize_features(df: pd.DataFrame) -> np.ndarray:
    """
    Normalize the selected features using StandardScaler.

    :param df: DataFrame of selected features
    :return: Normalized NumPy array of features
    """
    
    scaler = StandardScaler()
    normalize_data = scaler.fit_transform(df)
    return normalize_data 


In [190]:
# The function below is used to apply K-Means clustering on the data with 6 clusters
#help(KMeans)

def apply_kmeans(data: np.ndarray, n_clusters: int = 6) -> tuple:
    """
    Apply KMeans clustering to the normalized transaction data.

    :param data: Normalized feature array
    :param n_clusters: Number of clusters
    :return: Tuple (trained model, labels)
    """
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit( data)
    return kmeans, kmeans.labels_

#type(apply_kmeans(normalize_select_features_df))

In [191]:
# This function is used to add metrics to the data, such as:
# - The distance from each point to its cluster centroid
# - The distance as a percentage, which helps us understand how "far" a point is compared to others this is useful especially when we are not domain experts
# I also implemented the Modified Z-Score, which takes into account the density of the cluster.
# 
# The Modified Z-Score is useful for detecting outliers. It tells us how far a point is from the median
# considering the spread using Median Absolute Deviation instead of standard deviation
# It works better than the standard Z-Score when data is not normally distributed or has outliers it's the case there
# I was inspired by this article : https://www.kaggle.com/code/praxitelisk/anomaly-detection-techniques-summary


def add_distance_centroid_zscore(data_kmeans: KMeans, data: np.ndarray) -> pd.DataFrame:
    """
    Adds to each point:
    - Euclidean distance to the centroid,
    - distance to the centroid as a percentage of the cluster's maximum length,
    - a z-score based on the distance to the centroid computed within each cluster.

    :param data_kmeans: Trained KMeans model
    :param data: Input data (n_samples, n_features) used for clustering
    :return: DataFrame enriched with cluster-specific metrics
    """
    
    cluster_labels = data_kmeans.labels_
    centroids = data_kmeans.cluster_centers_
    n_clusters = data_kmeans.n_clusters

    # Compute Euclidean distances from each point to its cluster's centroid
    distances_to_centroid = np.linalg.norm(data - centroids[cluster_labels], axis=1)
    percent_distances = np.zeros(len(data))
    zscores = np.zeros(len(data))

    for cluster_id in range(n_clusters):
        # Mask for points in the current cluster
        mask = cluster_labels == cluster_id
        cluster_distances = distances_to_centroid[mask]
        max_dist = cluster_distances.max()

        # Calculate distance in percentage
        if max_dist > 0:
            percent_distances[mask] = (cluster_distances / max_dist) * 100
        else:
            percent_distances[mask] = 0

        # Calculate modified z-scores: z = 0.6745 * (distance - median) / MAD (if MAD > 0)
        median_d = np.median(cluster_distances)
        mad = np.median(np.abs(cluster_distances - median_d))
        if mad > 0:
            zscores[mask] = 0.6745 * (cluster_distances - median_d) / mad
        else:
            zscores[mask] = 0

    # Build the enriched DataFrame
    df = pd.DataFrame(data, columns=[f"data_col_{i}" for i in range(data.shape[1])])
    df['cluster_label'] = cluster_labels
    df['distance_to_centroid'] = distances_to_centroid
    df['distance_percent'] = percent_distances
    df['zscore'] = zscores

    return df

In [192]:
# Based on the previous function, I can now specify a threshold in the form of a maximum distance, 
# as well as percentage and z-score limits, to identify an element as an anomaly.

def detect_anomalies(df, methods=['distance_percent', 'zscore'], thresholds=None):
    """
    Detect anomalies using simple thresholding on:
    - distance_percent: relative distance to cluster center
    - zscore: standard score within cluster
    - distance_to_centroid: raw Euclidean distance

    Parameters:
        df (DataFrame): Must contain the relevant columns.
        methods (list): List of methods to use.
        thresholds (dict): Dictionary of thresholds for each method.

    Returns:
        np.ndarray: Boolean mask of anomalies.
    """
    if thresholds is None:
        thresholds = {
            'distance_percent': 95,
            'zscore': 4,  # Points with a zscore above 4 considered anomalous
        }

    anomalies = np.zeros(len(df), dtype=bool)

    if 'distance_percent' in methods:
        anomalies |= df['distance_percent'] > thresholds['distance_percent']

    if 'zscore' in methods:
        anomalies |= df['zscore'] > thresholds['zscore']

    return anomalies

In [193]:
# Written with the assistance of Copilot but implementation has been reviewed and understood.

def plot_clusters(
    normalized_data: np.ndarray,
    labels: np.ndarray,
    original_df: pd.DataFrame,
    anomalies_mask: np.ndarray = None,
    sample_size: int = 5000
):
    """
    Plot 2D PCA clusters, highlight anomalies by surrounding their markers, and show original (non-normalized) values in hover.

    Parameters:
      - normalized_data: standardized feature array (n_samples, n_features)
      - labels: cluster labels (n_samples,)
      - original_df: DataFrame (n_samples, m) with original columns to display in hover
      - anomalies_mask: boolean array (n_samples,) marking anomalies
      - sample_size: maximum points to plot
    """

    # PCA → 2D
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(normalized_data)

    # Build DataFrame for plotting
    df = pd.DataFrame(reduced, columns=["PC1", "PC2"])
    df["cluster"] = labels.astype(str)
    df["anomaly"] = anomalies_mask if anomalies_mask is not None else False
    # Attach original (non-normalized) columns
    df = pd.concat([df, original_df.reset_index(drop=True)], axis=1)

    # Sampling if too many points
    if len(df) > sample_size:
        df = df.sample(sample_size, random_state=42)

    # Base scatter: display all points colored by cluster, with hover showing the original columns, cluster, anomaly, PC1, and PC2.
    hover_cols = list(original_df.columns) + ["cluster", "anomaly"]
    fig = px.scatter(
        df,
        x="PC1",
        y="PC2",
        color="cluster",
        symbol="anomaly",
        symbol_map={False: "circle", True: "circle"},
        hover_data=hover_cols,
        opacity=0.6,
        title=f"PCA Clusters (n={len(df)})",
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    fig.update_traces(marker=dict(size=6))

    # Overlay anomalies: add an extra trace for points identified as anomalies
    # These anomalies will use the same color as their cluster but with a border (here in black) to distinguish them
    if anomalies_mask is not None:
        anom_df = df[df["anomaly"] == True]
        if not anom_df.empty:
            # Create a cluster-to-color mapping using Plotly Express’s color sequence
            clusters = sorted(df["cluster"].unique())
            colors = px.colors.qualitative.Set1
            color_map = {clust: colors[i % len(colors)] for i, clust in enumerate(clusters)}

            # For each cluster in the anomalies, add a trace with a border
            for clust in anom_df["cluster"].unique():
                sub = anom_df[anom_df["cluster"] == clust]
                # Hover logic
                cols = list(original_df.columns)
                hover_lines = [f"{col}: %{{customdata[{i}]}}" for i, col in enumerate(cols)]
                hover_lines += ["cluster: %{text}", "anomaly: True", "PC1: %{x}", "PC2: %{y}"]
                hovertemplate = "<br>".join(hover_lines) + "<extra></extra>"

                fig.add_trace(go.Scatter(
                    x=sub["PC1"],
                    y=sub["PC2"],
                    mode="markers",
                    marker=dict(
                        symbol="circle",
                        size=6, # slightly larger size to highlight the anomaly
                        color=color_map[clust],
                        line=dict(width=3, color="black")# black border to outline
                    ),
                    name=f"Anomalies Cluster {clust}",
                    text=[clust] * len(sub),
                    customdata=sub[cols].to_numpy(),
                    hovertemplate=hovertemplate
                ))

    # Centroids: calculate and display the centroids projected into PCA space
    centroids = np.array([normalized_data[labels == k].mean(axis=0) for k in np.unique(labels)])
    centroids_2d = pca.transform(centroids)
    fig.add_trace(go.Scatter(
        x=centroids_2d[:, 0],
        y=centroids_2d[:, 1],
        mode="markers+text",
        marker=dict(symbol="diamond", size=12, color="black", line=dict(color="white", width=1)),
        text=[f"C{k}" for k in np.unique(labels)],
        textposition="top center",
        name="Centroids"
    ))

    fig.show()


### Step 1: Loading the Data  
Objective: Load the transactions.csv file containing 20,000 transactions.

In [194]:
file_path = "transactions.csv"
data_as_dataframe = load_data(file_path)
data_as_dataframe.head(4)

,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date
0,1,417,583,102.332066,11.004279,11.004279,91.327787,2021-10-07 12:16:55.521706
1,2,68,277,83.130697,6.325441,6.325441,76.805256,2023-08-16 00:26:54.521999
2,3,621,546,96.091707,2.435575,2.435575,93.656132,2024-03-26 11:50:51.518583
3,4,783,337,82.476615,13.056381,13.056381,69.420234,2025-01-30 13:57:10.355813


Data Exploration

In [195]:
#help(data_as_dataframe)
print(data_as_dataframe.shape)
print(data_as_dataframe.columns)
data_as_dataframe.describe()

(20000, 8)
Index(['global_id', 'sender_id', 'receiver_id', 'initial_amount',
       'transfer_amount', 'amount_received', 'final_amount', 'date'],
      dtype='object')


,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,570.378550,490.870500,35458.260148,6553.204543,6540.375758,28925.153352
std,5773.647028,339.093935,289.824756,78367.259754,11994.880623,12001.848158,67993.367313
min,1.000000,1.000000,1.000000,22.459093,-7110.587943,-7110.587943,-21844.157920
25%,5000.750000,275.750000,241.000000,92.533501,57.031151,6.165345,36.790250
50%,10000.500000,547.000000,487.000000,160.364822,101.641698,101.641698,74.531949
75%,15000.250000,880.000000,741.000000,2015.712188,6959.000000,6959.000000,121.562098
max,20000.000000,1098.000000,999.000000,375075.838829,64525.609390,64525.609390,353664.953038


In [196]:
print(data_as_dataframe.isnull().sum())

global_id          0
sender_id          0
receiver_id        0
initial_amount     0
transfer_amount    0
amount_received    0
final_amount       0
date               0
dtype: int64


#### Step 2: Selecting Relevant Features  
Objective: Choose the useful columns for anomaly detection (e.g., initial_amount, transfer_amount, amount_received, final_amount).

In [197]:
select_features_df = select_features(data_as_dataframe)
select_features_df.head()

,initial_amount,transfer_amount,amount_received,final_amount
0,102.332066,11.004279,11.004279,91.327787
1,83.130697,6.325441,6.325441,76.805256
2,96.091707,2.435575,2.435575,93.656132
3,82.476615,13.056381,13.056381,69.420234
4,92.090839,19.036857,19.036857,73.053982


#### Step 3: Data Preparation and Normalization  
Objective: Normalize the data before applying K-means.

In [198]:
#data_as_dataframe.describe()
normalize_select_features_df = normalize_features(select_features_df)

#### Step 4: Clustering with K-means  
Objective: Group the transactions into 6 clusters.

In [199]:
# from sklearn.metrics import silhouette_score
# scores = [silhouette_score(normalize_select_features_df, KMeans(n_clusters=k).fit_predict(normalize_select_features_df)) 
#           for k in range(2,11)]

Please sir, I checked different values of K-Means clusters to see the silhouette, and it appears that 2 is better than 6, but I will continue with 6 because the exercise says so.

In [200]:
# scores

In [201]:
data_kmean, data_kmean_labels = apply_kmeans(data=normalize_select_features_df, n_clusters=6)

print("data_kmean_labels  :  ",data_kmean_labels,"\n")
print("len(data_kmean_labels) : ",len(data_kmean_labels),"\n")
print("data_kmean.cluster_centers_ : ",data_kmean.cluster_centers_,"\n")
print("type(data_kmean) : ",type(data_kmean))

data_kmean_labels  :   [0 0 0 ... 2 2 4] 

len(data_kmean_labels) :  20000 

data_kmean.cluster_centers_ :  [[-0.44749743 -0.51556647 -0.5157902  -0.42467605]
 [ 1.85699545  3.08434778  3.08362613  1.59590806]
 [ 1.50652988  1.79474812  1.79477513  1.41947273]
 [ 2.68246817  2.25366302  2.25342361  2.69386809]
 [ 2.31805393  0.97771242  0.97821375  2.49894805]
 [-0.4241223  -0.0103489  -0.00927396 -0.48730145]] 

type(data_kmean) :  <class 'sklearn.cluster._kmeans.KMeans'>


#### Step 5: Anomaly Detection  
Objective: Identify abnormal transactions based on their distance from the cluster centroid.

In [202]:
# Update to use add_distance_centroid_zscore
analysis_kmeans = add_distance_centroid_zscore(data_kmean, normalize_select_features_df)

# Technical prints: print the highest and lowest zscore values for inspection
print("Highest zscore:", analysis_kmeans['zscore'].max())
print("Lowest zscore:", analysis_kmeans['zscore'].min())

#print(analysis_kmeans.head(20))

Highest zscore: 587.2402723261116
Lowest zscore: -3.3055770340839827


I display the analysis results and the points closest to and farthest from each centroid.
This helps me understand the characteristics that make each cluster distinct.


In [203]:
mask = detect_anomalies(analysis_kmeans, methods=['zscore'], thresholds={'zscore':4})
analysis_kmeans['anomaly'] = mask

# Fusionner avec les données originales pour retrouver les vraies colonnes ---
df = data_as_dataframe.copy()
df = df.join(
    analysis_kmeans[['cluster_label','distance_to_centroid','distance_percent','zscore','anomaly']],
    how='left'
)

# Total et par cluster (juste pour info) ---
print("Total anomalies :", df['anomaly'].sum())
print("Anomalies per cluster:")
print(df[df['anomaly']].groupby('cluster_label').size(), "\n")

# Pour chaque cluster : 4 points les plus proches et 4 points les plus éloignés du centroïde ---
centroids = data_kmean.cluster_centers_

for k in range(len(centroids)):
    sub = df[df['cluster_label']==k]
    if sub.empty:
        print(f"Cluster {k} empty\n")
        continue

    # 4 plus proches
    closest4 = sub.nsmallest(4, 'distance_to_centroid')
    # 4 plus éloignés
    farthest4 = sub.nlargest(4, 'distance_to_centroid')
    
    print(f"Cluster {k}:")
    print("  4 closest to centroid:")
    display(closest4)      # affiche le DataFrame interactif dans le notebook
    print("  4 farthest from centroid:")
    display(farthest4)     # idem
    print("\n")


Total anomalies : 1849
Anomalies per cluster:
cluster_label
0    1783
1      13
2      12
3      15
4      15
5      11
dtype: int64 

Cluster 0:
  4 closest to centroid:


,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date,cluster_label,distance_to_centroid,distance_percent,zscore,anomaly
3500,3501,356,751,45.144614,134.271529,134.271529,89.126915,2021-05-11 00:32:07.300663,0,0.026957,1.202087,-3.268438,False
4483,4484,632,784,50.110204,134.148657,134.148657,84.038452,2022-12-21 01:41:24.383284,0,0.026960,1.202200,-3.267760,False
7019,7020,163,341,50.065762,132.943028,132.943028,82.877266,2023-08-11 06:46:51.645093,0,0.027100,1.208439,-3.230472,False
7003,7004,765,197,49.380733,132.673452,132.673452,83.292718,2024-05-12 13:48:17.351610,0,0.027133,1.209903,-3.221719,False


  4 farthest from centroid:


,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date,cluster_label,distance_to_centroid,distance_percent,zscore,anomaly
17327,17328,1001,189,113027.154873,-2220.211278,-2220.211278,115247.366151,2023-11-14 18:50:31.734950,0,2.242537,100.000000,587.240272,True
19129,19130,1049,559,115075.034063,3837.467771,3837.467771,111237.566293,2024-07-24 00:43:03.669678,0,2.232485,99.551753,584.561131,True
17922,17923,1064,121,100474.280252,-7110.587943,-7110.587943,107584.868196,2024-02-17 19:47:03.428350,0,2.215360,98.788081,579.996709,True
18517,18518,1058,116,100299.442520,3813.095646,3813.095646,96486.346874,2024-09-25 04:40:32.942217,0,1.950082,86.958725,509.293417,True




Cluster 1:
  4 closest to centroid:


...

,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date,cluster_label,distance_to_centroid,distance_percent,zscore,anomaly
17340,17341,1001,778,361024.586584,7359.633545,7359.633545,353664.953038,2024-04-01 09:32:00.228594,4,3.196115,100.000000,7.285467,True
17969,17970,1064,546,249034.641204,-3973.091310,-3973.091310,253007.732514,2024-06-22 14:02:04.172401,4,2.771476,86.713894,6.004840,True
19445,19446,1014,698,141150.750030,-2398.430825,-2398.430825,143549.180855,2024-04-09 14:09:08.647445,4,2.746229,85.923962,5.928700,True
19908,19909,1082,580,325268.592370,6749.803221,6749.803221,318518.789149,2024-09-09 19:23:23.383061,4,2.617379,81.892503,5.540114,True




Cluster 5:
  4 closest to centroid:


,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date,cluster_label,distance_to_centroid,distance_percent,zscore,anomaly
12726,12727,621,667,2031.242128,6687.0,6687.0,-4655.757872,2021-12-08 00:00:00.000000,5,0.031204,1.503473,-3.305577,False
12713,12714,621,667,2021.790341,6687.0,6687.0,-4665.209659,2021-10-30 00:00:00.000000,5,0.031243,1.505367,-3.304891,False
12719,12720,621,667,2016.552492,6687.0,6687.0,-4670.447508,2021-11-17 00:00:00.000000,5,0.031266,1.506438,-3.304503,False
12723,12724,621,667,2015.390544,6687.0,6687.0,-4671.609456,2021-11-29 00:00:00.000000,5,0.031270,1.506678,-3.304417,False


  4 farthest from centroid:


,global_id,sender_id,receiver_id,initial_amount,transfer_amount,amount_received,final_amount,date,cluster_label,distance_to_centroid,distance_percent,zscore,anomaly
17785,17786,1034,960,107006.581583,11616.219889,11616.219889,95390.361694,2024-06-16 21:39:11.670894,5,2.075460,100.000000,32.367301,True
19402,19403,1014,241,36770.390102,22837.782002,22837.782002,13932.608100,2024-07-17 06:47:19.452483,5,2.001568,96.439738,31.077867,True
17106,17107,1069,155,92658.029236,13613.660805,13613.660805,79044.368431,2024-12-13 09:17:47.178542,5,1.883666,90.758985,29.020446,True
18617,18618,1058,303,80574.514053,16310.999563,16310.999563,64263.514490,2023-06-21 13:53:10.897560,5,1.835904,88.457719,28.186987,True


In [204]:
# ## Distribution of Anomaly Metrics
# Here we plot for each metric:
# 1. A histogram of all values  
# 2. A box‑and‑whiskers diagram (boxplot) broken out by cluster_label  
# This helps see the global shape of the distribution and compare clusters.

# select only the columns we need
# metrics = ['distance_to_centroid', 'distance_percent', 'zscore']

# for metric in metrics:
#     # Histogram of the metric
#     fig_hist = px.histogram(
#         df,
#         x=metric,
#         nbins=50,
#         title=f"Histogram of {metric}",
#         marginal="rug",              # add rug plot
#         opacity=0.7
#     )
#     fig_hist.update_layout(xaxis_title=metric, yaxis_title="Count")
#     fig_hist.show()

#     # Box‑and‑whiskers by cluster
#     fig_box = px.box(
#         df,
#         x='cluster_label',
#         y=metric,
#         points="all",                # show all points
#         title=f"Boxplot of {metric} by Cluster",
#     )
#     fig_box.update_layout(xaxis_title="Cluster Label", yaxis_title=metric)
#     fig_box.show()


In [205]:
anomalies = analysis_kmeans['anomaly'].values
plot_clusters(
  normalized_data=normalize_select_features_df,
  labels=data_kmean.labels_,
  original_df=select_features_df,
  anomalies_mask=anomalies,
  sample_size=20000
)